# Calculate a trailing stop loss using the ATR indicator

## Imports

In [ ]:
# %load_ext dotenv
# %dotenv ../.env

# %load_ext autoreload
# %autoreload 2


In [ ]:
import logging

from IPython.display import display, Markdown, HTML

from investmentstk.figures import candlestick
from investmentstk.figures.common import show_figure
from investmentstk.formulas.average_true_range import atr_stop_loss_from_asset
from investmentstk.models.asset import Asset
from investmentstk.strategy.brito_trend_following import PERIODICITY_PER_BROKER
from investmentstk.utils.calendar import is_last_bar_closed

## Input

In [ ]:
assets = "AV:5442,AV:5537,CMC:X-ABDOR,CMC:X-AAOTG".split(",")
show_all = False

In [ ]:
display(HTML('<style>div.output_area .rendered_html table.dataframe { margin: auto; }</style>'))

logging.getLogger("investmentstk").setLevel(logging.CRITICAL)

# Set this to True when running locally to get interactive charts
# When publishing the notebook on GitHub, we need static images
IS_INTERACTIVE = True

for asset in assets:
    asset = Asset.from_id(asset)
    
    # Hide asset if it's not time to update it's stop loss
    if not show_all:
        resolution = PERIODICITY_PER_BROKER[asset.source]
        if not is_last_bar_closed(resolution):
            continue

    display(Markdown(f'# {asset.name}'))

    dataframe = atr_stop_loss_from_asset(asset)
    dataframe = dataframe[-100:]

    figure = candlestick.generate_figure(dataframe, asset)
    figure.update_layout(width=950, height=600)
    show_figure(figure, interactive=IS_INTERACTIVE)

    dataframe['stop_change'] = dataframe['stop'] - dataframe['stop'].shift(1)
    dataframe['stop_change_pct'] = dataframe['stop_change'] / dataframe['close'] * 100
    dataframe.loc[dataframe['stop_change'] > 0, 'stop_change_direction'] = '↗️️'
    dataframe.loc[dataframe['stop_change'] < 0, 'stop_change_direction'] = '↘️️'
    dataframe.loc[dataframe['stop_change'] == 0, 'stop_change_direction'] = '=️️'
    
    dataframe = dataframe.round({'atr': 3, 'stop_distance': 2, 'stop_change': 2, 'stop_change_pct': 2})
    
    dataframe['stop_change'] = dataframe['stop_change'].astype(str)
    dataframe['stop_change'].replace(['0', '0.0'], '', inplace=True)
    
    dataframe['stop_change_pct'] = dataframe['stop_change_pct'].astype(str)
    dataframe['stop_change_pct'].replace(['0', '0.0'], '', inplace=True)

    display(dataframe.drop(['open', 'high', 'low'], axis=1).tail())